In [ ]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_images, plot_heatmap

In [ ]:
visual_path = os.path.join("benchmarks", "visual")

baseline_file_path = os.path.join(visual_path, "screenshot_baseline.png")

In [ ]:
baseline_image_bgr = cv2.imread(baseline_file_path)
baseline_image_rgb = cv2.cvtColor(baseline_image_bgr, cv2.COLOR_BGR2RGB)
baseline_image_gray = cv2.cvtColor(baseline_image_bgr, cv2.COLOR_BGR2GRAY)
print(baseline_image_rgb.shape)

plt.imshow(baseline_image_rgb)
plt.show()

In [ ]:
benchmark_filenames = {}

for dir_name in os.listdir(visual_path):
    sub_dir_path = os.path.join(visual_path, dir_name)
    if (os.path.isdir(sub_dir_path)):
        benchmark_filenames[dir_name] = os.listdir(sub_dir_path)

In [ ]:
benchmark_images = {}

for method, filenames in benchmark_filenames.items():
    folder_path = os.path.join(visual_path, method)

    benchmark_images[method] = []
    for filename in filenames:
        if (filename.split(".")[-1] == "png"):
            benchmark_file_path = os.path.join(folder_path, filename)
            benchmark_image_bgr = cv2.imread(benchmark_file_path)
            max_iteration_count = int((os.path.basename(benchmark_file_path).split(".")[-2]).split("_")[-1])
            benchmark_images[method].append((max_iteration_count, benchmark_image_bgr))

    benchmark_images[method].sort(key=lambda x: x[0])

benchmark_images = dict(sorted(benchmark_images.items(), key=lambda x: x[0])[::-1])

In [ ]:
benchmark_images_rgb = {}
benchmark_images_ssim_diff_gray = {}
ssim_scores = {}
max_iteration_counts = set()

for y, (method, images) in enumerate(benchmark_images.items()):
    benchmark_images_rgb[method] = []
    benchmark_images_ssim_diff_gray[method] = []
    ssim_scores[method] = []

    for x, (max_iteration_count, image_bgr) in enumerate(images):
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

        (ssim_score, image_ssim_diff_gray) = ssim(baseline_image_gray, image_gray, full=True, data_range=10)
        
        benchmark_images_rgb[method].append(image_rgb)
        benchmark_images_ssim_diff_gray[method].append(image_ssim_diff_gray)
        ssim_scores[method].append(round(ssim_score, 3))
        max_iteration_counts.add(max_iteration_count)

max_iteration_counts = sorted(max_iteration_counts)

In [ ]:
plot_images(
    images=np.array(list(benchmark_images_rgb.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(benchmark_images_rgb.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    figsize=15,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="Tracing methods at different max iteration counts"
)

In [ ]:
plot_images(
    images=np.array(list(benchmark_images_ssim_diff_gray.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(benchmark_images_ssim_diff_gray.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    figsize=15,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="SSIM error of different tracing methods at different max iteration counts compared to baseline image"
)

In [ ]:
plot_heatmap(
    matrix=np.array(list(ssim_scores.values())),
    row_labels=[" ".join(str(i).split("_")) for i in list(ssim_scores.keys())],
    col_labels=[str(i) for i in list(max_iteration_counts)],
    meassurment_unit="",
    x_figsize=15,
    y_figsize=6,
    x_label="Max iteration count",
    y_label="Tracing method",
    title="SSIM scores compared to baseline image"
)